# Qubit-qumode circuit synthesis

Using optimizers based on `NumPy` objects to present qubit-qumode or qmode circuit for a given qumode unitary operator.

## Prerequisite

Installation cells for Google Colab users.

In [1]:
!pip install qutip
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 31.4 MB/s eta 0:00:00


Import libaries.

In [1]:
import numpy as np
import qutip as qt
import scipy.optimize as sciopt

from scipy.linalg import expm
from functools import partial
from itertools import combinations

In [2]:
import sys

In [3]:
import matplotlib.pyplot as plt

## Ansatz circuits

We will using `QuTip` and follow ( qubit $\otimes$ qumode ) notation where applicable.

### Basics

In [4]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [5]:
def qproj00():
    return qt.basis(2, 0).proj()

def qproj11():
    return qt.basis(2, 1).proj()

def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)

def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)

In [6]:
def qubit_rot(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( qt.sigmax() * np.cos(phi) )
    gen += ( qt.sigmay() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return H.expm()

### ECD-rotation

Qubit rotation with qumode echoed conditional displacement (ECD) operator ([reference](https://doi.org/10.1038/s41567-022-01776-9))

\begin{align*}
U (\beta, \theta, \phi)
&= ECD (\beta) \: \big[ R (\theta, \phi) \otimes I \big],
\\
R (\theta, \phi)
&= e^{ - i (\theta / 2) \big[ \cos(\phi) X + \sin(\phi) Y \big] },
\\
ECD (\beta)
&= |1 \rangle \langle 0| \otimes D (\beta / 2)  
+ |0 \rangle \langle 1| \otimes D (-\beta / 2),
\\
D (\beta)
&= e^{ \beta a^\dagger - \beta^* a }.
\end{align*}

In [7]:
def ecd_rot_op(beta, theta, phi, nfock):
    """
    ECD-rotation operator.

    Arguments:
    beta -- ECD parameter
    theta, phi -- rotation parameters
    nfock -- Fock cutoff
    """
    # ECD
    E2 = qt.tensor(qproj10(), qt.displace(nfock, beta/2))
    E2 += qt.tensor(qproj01(), qt.displace(nfock, -beta/2))

    # Rotation
    R2 = qt.tensor(qubit_rot(theta, phi), qt.qeye(nfock))

    return E2 * R2

In [36]:
qt.displace(2, 0.1/2).full()

array([[ 0.99875026+0.j, -0.04997917+0.j],
       [ 0.04997917+0.j,  0.99875026+0.j]])

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\beta}, \vec{\theta}, \vec{\phi})
= U (\beta_{N_d}, \theta_{N_d}, \phi_{N_d}) \cdots
U (\beta_1, \theta_1, \phi_1),
$$

where $\vec{\beta}, \vec{\theta}$, and $\vec{\phi}$ are $N_d$-dimensional vectors.

In [8]:
def ecd_rot_ansatz(bmagvec, bargvec, thetavec, phivec, nfock):
    """
    ECD-rotation ansatz.

    Arguments:
    bmagvec, bargvec -- ECD parameters
    thetavec, phivec -- rotation parameters
    nfock -- Fock cutoff
    """
    # Check
    if bmagvec.shape != bargvec.shape:
        raise ValueError("Dimensions of bmagvec and bargvec do not match.")
    betavec = get_cvec_np(bmagvec, bargvec)
    if betavec.shape != thetavec.shape:
        raise ValueError("Lengths of betavec and thetavec do not match.")
    if betavec.shape != phivec.shape:
        raise ValueError("Lengths of betavec and phivec do not match.")

    # Initialize
    ndepth = betavec.shape[0]
    uni = ecd_rot_op(betavec[0], thetavec[0], phivec[0], nfock)

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = ecd_rot_op(betavec[i], thetavec[i], phivec[i], nfock)
        uni = ( new_uni * uni )

    return uni.full()

### SQR-displacement

Photon-number selective qubit rotation (SQR) and displacement operator ([reference](https://arxiv.org/abs/2407.10381))

\begin{align*}
U (\beta, \vec{\theta}, \vec{\phi})
&= SQR (\vec{\theta}, \vec{\phi}) \:
\big[ I \otimes D (\beta) \big],
\\
SQR (\vec{\theta}, \vec{\phi})
&= \sum_{n = 0}^{L - 1} \: R ( \theta_n, \phi_n ) \otimes
|n \rangle \langle n|,
\\
R (\theta, \phi)
&= e^{ - i (\theta / 2) \big[ \cos(\phi) X + \sin(\phi) Y \big] },
\\
D (\beta)
&= e^{ \beta a^\dagger - \beta^* a }.
\end{align*}

In [9]:
def sqr_disp_op(beta, thetavec, phivec):
    """
    SQR-displacement operator.

    Arguments:
    beta -- displacement coefficient
    thetavec, phivec -- SQR parameters
    """
    # Check
    if thetavec.shape != phivec.shape:
        raise ValueError("Lengths of thetavec and phivec do not match.")

    # Initialize
    nfock = thetavec.shape[0]

    # SQR
    S2 = qt.tensor( qubit_rot(thetavec[0], phivec[0]), qt.basis(nfock, 0).proj() )
    for i in range(1, nfock):
        S2 += qt.tensor( qubit_rot(thetavec[i], phivec[i]), qt.basis(nfock, i).proj() )

    # Displacement
    D2 = qt.tensor(qt.qeye(2), qt.displace(nfock, beta))

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\beta}, \bar{\theta}, \bar{\phi})
= U (\beta_{N_d}, \vec{\theta}_{N_d}, \vec{\phi}_{N_d}) \cdots
U (\beta_1, \vec{\theta}_1, \vec{\phi}_1),
$$

where $\vec{\beta}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L}, \bar{\phi}_{N_d \times L} $ are matrices.


In [10]:
def sqr_disp_ansatz(bmagvec, bargvec, thetamat, phimat, nfock):
    """
    SQR-displacement ansatz.

    Arguments:
    bmagvec, bargvec -- displacement parameters
    thetamat, phimat -- SQR parameters
    nfock -- Fock cutoff
    """
    # Check
    if bmagvec.shape != bargvec.shape:
        raise ValueError("Dimensions of bmagvec and bargvec do not match.")
    betavec = get_cvec_np(bmagvec, bargvec)
    if thetamat.shape != phimat.shape:
        raise ValueError("Dimensions of thetamat and phimat do not match.")
    if thetamat.shape[0] != betavec.shape[0]:
        raise ValueError("Lengths of theta and beta do not match.")
    if thetamat.shape[1] != nfock:
        raise ValueError("Incorrect nfock chosen.")

    # Initialize
    ndepth = thetamat.shape[0]
    uni = sqr_disp_op(betavec[0], thetamat[0, :], phimat[0, :])

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = sqr_disp_op(betavec[i], thetamat[i, :], phimat[i, :])
        uni = ( new_uni * uni )

    return uni.full()

### SNAP-displacement

Selective number-dependent arbitray phase (SNAP) and displacement operator ([reference](https://doi.org/10.1103/PhysRevA.92.040303))

\begin{align*}
U (\alpha, \vec{\theta})
&= S (\vec{\theta}) \: D (\alpha),
\\
S (\vec{\theta})
&= \sum_{n = 0}^{L - 1} \: \exp ( i \: \theta_n ) \: |n \rangle \langle n|,
\\
D (\alpha)
&= e^{ \alpha \: ( a^\dagger - a ) }.
\end{align*}

In [11]:
def snap_disp_op(alpha, thetavec):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # Initialize
    nfock = thetavec.shape[0]

    # SNAP
    S2 = np.exp(1j * thetavec[0]) * qt.basis(nfock, 0).proj()
    for i in range(1, nfock):
        S2 += np.exp(1j * thetavec[i]) * qt.basis(nfock, i).proj()

    # Rotation
    D2 = qt.displace(nfock, alpha)

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\alpha}, \bar{\theta})
= U (\alpha_{N_d}, \vec{\theta}_{N_d}) \cdots
U (\alpha_1, \vec{\theta}_1),
$$

where $\vec{\alpha}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L} $ is a matrix.


In [12]:
def snap_disp_ansatz(alphavec, thetamat, nfock):
    """
    SNAP-displacement ansatz.

    Arguments:
    alphavec -- displacement parameters
    thetamat -- SNAP parameters
    nfock -- Fock cutoff
    """
    # Check
    if thetamat.shape[0] != alphavec.shape[0]:
        raise ValueError("Lengths of theta and alpha do not match.")
    if thetamat.shape[1] != nfock:
        raise ValueError("Incorrect nfock chosen.")

    # Initialize
    ndepth = thetamat.shape[0]
    uni = snap_disp_op(alphavec[0], thetamat[0, :])

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = snap_disp_op(alphavec[i], thetamat[i, :])
        uni = ( new_uni * uni )

    return uni.full()

## Cost functions

Let us denote $V_T$ as our target qumode matrix and $U$ is the parametrized matrix.
The cost functions are

\begin{align*}
C_{diag}
&= \frac{1}{L} \: \sum_{n = 0}^{L - 1} \:
| V_{n, n} - U_{n, n} |^2,
\\
C_{full}
&= \frac{1}{L^2} \: \sum_{n, m = 0}^{L - 1} \:
| V_{n, m} - U_{n, m} |^2,
\end{align*}
where $L$ is the qumode Fock cutoff.

If $U$ is a qubit-qumode circuit then

\begin{align*}
V_{n, m}
&= \langle 0, n| ( I \otimes V_T ) | 0, m \rangle,
\\
U_{n, m}
&= \langle 0, n| U | 0, m \rangle,
\end{align*}
and if $U$ is a qumode circuit then

\begin{align*}
V_{n, m}
&= \langle n| V_T | m \rangle,
\\
U_{n, m}
&= \langle n| U | m \rangle.
\end{align*}

### Basics

In [13]:
def tran_val_qubit_cavity(op, n, m):
    """
    Compute <0, n| O |0, m>.

    Arguments:
    op -- Operator matrix
    n -- Fock level
    """
    # Check
    L = op.shape[0] // 2
    if n > L:
        raise ValueError("n > L.")
    if m > L:
        raise ValueError("m > L.")

    # |0, n> and |0, m>
    state1 = qt.tensor(qt.basis(2, 0), qt.basis(L, n)).full()
    state2 = qt.tensor(qt.basis(2, 0), qt.basis(L, m)).full()

    # <0, n| O |0, m>
    t1 = np.matmul(op, state2)
    ov = np.dot(np.conj(state1).T, t1)

    return np.squeeze(ov)

In [14]:
def tran_val_only_cavity(op, n, m):
    """
    Compute <n| O |m>.

    Arguments:
    op -- Operator matrix
    n -- Fock level
    """
    # Check
    L = op.shape[0]
    if n > L:
        raise ValueError("n > L.")
    if m > L:
        raise ValueError("m > L.")

    # |n> and |m>
    state1 = qt.basis(L, n).full()
    state2 = qt.basis(L, m).full()

    # <n| O |m>
    t1 = np.matmul(op, state2)
    ov = np.dot(np.conj(state1).T, t1)

    return np.squeeze(ov)

### ECD-rotation

In [15]:
def unpack_params_ecd_rot(X):
    # Initialize
    dim = X.shape[0] // 4

    # Unpack
    beta_mag = X[:dim].copy()
    beta_arg = X[dim:2*dim].copy()
    theta = X[2*dim:3*dim].copy()
    phi = X[3*dim:4*dim].copy()

    return beta_mag, beta_arg, theta, phi

def pack_params_ecd_rot(beta_mag, beta_arg, theta, phi):
    # Initialize
    dim = beta_mag.shape[0]
    X = np.zeros((4 * dim,))

    # Pack
    X[:dim] = beta_mag.copy()
    X[dim:2*dim] = beta_arg.copy()
    X[2*dim:3*dim] = theta.copy()
    X[3*dim:4*dim] = phi.copy()

    return X

In [16]:
def cost_fun_ecd_rot(Xvec, V, ctype='full'):
    """
    Loss function.

    Arguments:
    Xvec -- ansatz parameters
    V -- target qubit-qumode operator
    ctype -- type of cost fucntion
    """
    # Unpack
    bmagvec, bargvec, thetavec, phivec = unpack_params_ecd_rot(Xvec)

    # ECD unitary
    nfock = V.shape[0] // 2
    U = ecd_rot_ansatz(bmagvec, bargvec, thetavec, phivec, nfock)

    # Initialize
    ov = 0.0

    # Diagonal
    if ctype == 'diag':
        for j in range(nfock):
            t0 = tran_val_qubit_cavity(V, j, j)
            t1 = tran_val_qubit_cavity(U, j, j)
            ov += np.abs( t0 - t1 )**2
        return ov / nfock

    # Full
    for j in range(nfock):
        for k in range(nfock):
            t0 = tran_val_qubit_cavity(V, j, k)
            t1 = tran_val_qubit_cavity(U, j, k)
            ov += np.abs( t0 - t1 )**2

    return ov / (nfock**2)

### SQR-displacement

In [17]:
def unpack_params_sqr_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (2 * (nfock + 1))

    # Unpack
    beta_mag = X[:ndepth].copy()
    beta_arg = X[ndepth:2*ndepth].copy()
    d1 = 2 * ndepth
    d2 = ndepth * nfock
    theta = X[d1:d1+d2].reshape((ndepth, nfock))
    phi = X[d1+d2:d1+2*d2].reshape((ndepth, nfock))

    return beta_mag, beta_arg, theta, phi

def pack_params_sqr_disp(beta_mag, beta_arg, theta, phi):
    # Initialize
    ndepth = beta_mag.shape[0]
    nfock = theta.shape[1]
    dim = 2 * (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = beta_mag.copy()
    X[ndepth:2*ndepth] = beta_arg.copy()
    d1 = 2 * ndepth
    d2 = ndepth * nfock
    X[d1:d1+d2] = theta.reshape(-1)
    d3 = d1 + d2
    X[d3:d3+d2] = phi.reshape(-1)

    return X

In [18]:
def cost_fun_sqr_disp(Xvec, V, ctype='full'):
    """
    Loss function.

    Arguments:
    Xvec -- ansatz parameters
    V -- target qubit-qumode operator
    ctype -- type of cost fucntion
    """
    # Unpack
    nfock = V.shape[0] // 2
    bmagvec, bargvec, thetamat, phimat = unpack_params_sqr_disp(Xvec, nfock)

    # ECD unitary
    U = sqr_disp_ansatz(bmagvec, bargvec, thetamat, phimat, nfock)

    # Diagonal
    if ctype == 'diag':
        ov = 0.0
        for j in range(nfock):
            t0 = tran_val_qubit_cavity(V, j, j)
            t1 = tran_val_qubit_cavity(U, j, j)
            ov += np.abs( t0 - t1 )**2
        return ov / nfock

    # Full
    ov = 0.0
    for j in range(nfock):
        for k in range(nfock):
            t0 = tran_val_qubit_cavity(V, j, k)
            t1 = tran_val_qubit_cavity(U, j, k)
            ov += np.abs( t0 - t1 )**2
    ov *= ( 1 / ( nfock**2 ) )

    return ov

### SNAP-displacement

In [19]:
def unpack_params_snap_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (nfock + 1)

    # Unpack
    alpha = X[:ndepth].copy()
    d1 = ndepth * nfock
    theta = X[ndepth:ndepth+d1].reshape((ndepth, nfock))

    return alpha, theta

def pack_params_snap_disp(alpha, theta):
    # Initialize
    ndepth = alpha.shape[0]
    nfock = theta.shape[1]
    dim = (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = alpha.copy()
    d1 = ndepth * nfock
    X[ndepth:ndepth+d1] = theta.reshape(-1)

    return X

In [20]:
def cost_fun_snap_disp(Xvec, V, ctype='full'):
    """
    Loss function.

    Arguments:
    Xvec -- ansatz parameters
    V -- target qumode operator
    ctype -- type of cost fucntion
    """
    # Unpack
    nfock = V.shape[0]
    alphavec, thetamat = unpack_params_snap_disp(Xvec, nfock)

    # ECD unitary
    U = snap_disp_ansatz(alphavec, thetamat, nfock)

    # Initialize
    ov = 0.0

    # Diagonal
    if ctype == 'diag':
        for j in range(nfock):
            t0 = tran_val_only_cavity(V, j, j)
            t1 = tran_val_only_cavity(U, j, j)
            ov += np.abs( t0 - t1 )**2
        return ov / nfock

    # Full
    for j in range(nfock):
        for k in range(nfock):
            t0 = tran_val_only_cavity(V, j, k)
            t1 = tran_val_only_cavity(U, j, k)
            ov += np.abs( t0 - t1 )**2
    ov *= ( 1 / ( nfock**2 ) )

    return ov

## Optimizations

### Core

In [ ]:
def opt_scipy_ecd_rot(V, ndepth, ctype, maxiter=100, method='COBYLA', verb=0, \
                      threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    V -- target qumode matrix in QuTip
    ndepth -- ansatz circuit depth
    ctype -- cost function type
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- print additional things or not
    threshold -- error tolerance
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    # Qubit-qumode matrix
    # FullV = qt.tensor(qt.qeye(2), V).full()
    
    print(FullV)

    # Bound parameters
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([beta_mag_min, beta_mag_max])
    for _ in range(ndepth):
        bounds.append([beta_arg_min, beta_arg_max])
    for _ in range(ndepth):
        bounds.append([theta_min, theta_max])
    for _ in range(ndepth):
        bounds.append([phi_min, phi_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        beta_mag = np.random.uniform(0, 3, size=ndepth)
        beta_arg = np.random.uniform(0, np.pi, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=ndepth)
        phi = np.random.uniform(0, np.pi, size=ndepth)
        Xvec = pack_params_ecd_rot(beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(cost_fun_ecd_rot, V=FullV, ctype=ctype)

    # Intermediate values
    iteration_step = 0
    print_freq = 10
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {obj_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    return result.fun, result.x

In [38]:
def opt_scipy_sqr_disp(V, ndepth, ctype, maxiter=100, method='COBYLA', verb=0, \
                       threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    V -- target qumode matrix in QuTip
    ndepth -- ansatz circuit depth
    ctype -- cost function type
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- print additional things or not
    threshold -- error tolerance
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    """
    Minimize the cost function using Scikit-based methods.

    Arguments:
    V -- target qumode matrix in QuTip
    ndepth -- ansatz circuit depth
    ctype -- cost function type
    budget -- maximum number of iterations
    method -- optimization method
    Xvec -- optional initial guess
    """
    # Qubit-qumode matrix
    FullV = qt.tensor(qt.qeye(2), V).full()
    nfock = FullV.shape[0] // 2

    # Bound parameters
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([beta_mag_min, beta_mag_max])
    for _ in range(ndepth):
        bounds.append([beta_arg_min, beta_arg_max])
    for _ in range(ndepth * nfock):
        bounds.append([theta_min, theta_max])
    for _ in range(ndepth * nfock):
        bounds.append([phi_min, phi_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        beta_mag = np.random.uniform(0, 3, size=ndepth)
        beta_arg = np.random.uniform(0, np.pi, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        phi = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        Xvec = pack_params_sqr_disp(beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(cost_fun_sqr_disp, V=FullV, ctype=ctype)

    # Intermediate values
    iteration_step = 0
    print_freq = 10
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {obj_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    return result.fun, result.x

In [39]:
def opt_scipy_snap_disp(V, ndepth, ctype, maxiter=100, method='COBYLA', verb=0, \
                        threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    V -- target qumode matrix in QuTip
    ndepth -- ansatz circuit depth
    ctype -- cost function type
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- print additional things or not
    threshold -- error tolerance
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    # Qumode matrix
    FullV = V.full()
    nfock = FullV.shape[0]

    # Bound parameters
    alpha_min = 0.0
    alpha_max = 10.0
    theta_min = 0.0
    theta_max = np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([alpha_min, alpha_max])
    for _ in range(ndepth * nfock):
        bounds.append([theta_min, theta_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        alpha = np.random.uniform(0, 3, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        Xvec = pack_params_snap_disp(alpha, theta)

    # Loss function
    obj_fun = partial(cost_fun_snap_disp, V=FullV, ctype=ctype)

    # Intermediate values
    iteration_step = 0
    print_freq = 10
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {obj_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    return result.fun, result.x

### Wrapper

In [40]:
def qumode_opt_regular(V, ndepth, ctype, atype='ecd-rot', budget=100, method='COBYLA', \
                    verb=0, threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using various optimizers.

    Arguments:
    V -- target qumode matrix in QuTip
    ctype -- type of cost function
    atype -- type of ansatz
    ndepth -- ansatz circuit depth
    budget -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    Xvec -- optional initial guess
    """
    # Method library
    sc_tuple = ('COBYLA', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'trust-constr')

    # Optimize
    if atype == 'sqr-disp':
        if method in sc_tuple:
            loss, Xvec = opt_scipy_sqr_disp(V, ndepth, ctype=ctype, maxiter=budget, \
                                            method=method, verb=verb, \
                                            threshold=threshold, Xvec=Xvec)
        else:
            print(f"Error: method not found")
            sys.exit(1)
    elif atype == 'snap-disp':
        if method in sc_tuple:
            loss, Xvec = opt_scipy_snap_disp(V, ndepth, ctype=ctype, maxiter=budget, \
                                             method=method, verb=verb, \
                                             threshold=threshold, Xvec=Xvec)
        else:
            print(f"Error: method not found")
            sys.exit(1)
    else:
        if method in sc_tuple:
            loss, Xvec = opt_scipy_ecd_rot(V, ndepth, ctype=ctype, maxiter=budget, \
                                           method=method, verb=verb, \
                                           threshold=threshold, Xvec=Xvec)
        else:
            print(f"Error: method not found")
            sys.exit(1)

    return loss, Xvec

## Explore

In [41]:
def test_opt_uni(ndepth, atype):
    # Target
    V = qt.tensor( qt.sigmax(), qt.sigmaz() )
    print(V)

    # Optimize
    loss, Xvec = qumode_opt_regular(V, ndepth, ctype='full', atype=atype, budget=int(1e3), \
                                    method='BFGS', verb=0, threshold=1e-12)

    return loss, Xvec

In [44]:
ndepth = 5
atype = 'ecd-rot' #'ecd-rot' #'sqr-disp'

loss, Xvec = test_opt_uni(ndepth, atype)
loss

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]
 [ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]]
[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]]
         Current function value: 0.016360
         Iterations: 153
         Function evaluations: 3612
         Gradient evaluations: 172


/Users/zhen002/miniconda3/envs/qbos/lib/python3.12/site-packages/scipy/optimize/_minimize.py:726: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


0.016360263365860085

In [29]:
Xvec

array([ 0.83058848,  2.77807024,  0.89457258,  1.6245252 ,  0.38459129,
        2.28563568,  2.83942265,  2.11735481,  2.46064887,  1.13332207,
        2.04516356,  3.57978842,  1.43122665,  2.29743513,  3.42062083,
        0.98779065,  1.66764103,  0.39671077,  4.88654179, -0.51796771])

In [30]:
np.set_printoptions(suppress=True)
ecd_rot_ansatz(np.array([1]), np.array([0.2]), np.array([0.3]), np.array([0.4]), 2).data.as_ndarray()

array([[ 0.05107   -0.1207919j ,  0.01423356-0.07021634j,
         0.86772826+0.j        ,  0.46459283-0.09417763j],
       [-0.0404535 +0.05913072j,  0.05107   -0.1207919j ,
        -0.46459283-0.09417763j,  0.86772826+0.j        ],
       [ 0.86772826+0.j        , -0.46459283+0.09417763j,
        -0.05107   -0.1207919j ,  0.0404535 +0.05913072j],
       [ 0.46459283+0.09417763j,  0.86772826+0.j        ,
        -0.01423356-0.07021634j, -0.05107   -0.1207919j ]])